In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform
import time, os, datetime
from Model import Residual_Unit
from Model import Attention_Block
from Model import Attention_Block_NAL
from Model import AttentionResNet92
from Model import AttentionResNet92_NAL

/jet/var/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/jet/var/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/jet/var/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/jet/var/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: P

In [2]:
print("TF version: ",tf.__version__)
print("Keras version:",tf.keras.__version__)

TF version:  1.10.0
Keras version: 2.1.6-tf


In [3]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping,TensorBoard, LearningRateScheduler
from tensorflow.keras.models import load_model

In [4]:
# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train[:45000, :, :, :]
y_train = y_train[:45000]
x_val = x_train[-5000:, :, :, :]
y_val = y_train[-5000:]

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (45000, 32, 32, 3)
y_train shape: (45000, 1)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [5]:
# define generators for training and validation data
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

val_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)

test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)

# compute quantities required for feature normalization
train_datagen.fit(x_train)
val_datagen.fit(x_val)
test_datagen.fit(x_test)

In [6]:
model = AttentionResNet92(shape=(32,32,3), in_channel=32, kernel_size=5, n_classes=10, dropout=None, regularization=0.00001)
'''
def lr_schedule(epoch):
    lr = 1e-5
    if epoch > 80:
        lr *= 1e-5
    elif epoch > 60:
        lr *= 1e-4
    elif epoch > 40:
        lr *= 1e-2
    elif epoch >20:
        lr *= 1e-1
    print('Learning rate:', lr)
    return lr
'''

def lr_schedule(epoch):
    lr = 1e-5
    if epoch > 90:
        lr *= 1e-30
    elif epoch > 80:
        lr *= 1e-25
    elif epoch > 70:
        lr *= 1e-19
    elif epoch > 60:
        lr *= 1e-14
    elif epoch > 50:
        lr *= 1e-10
    elif epoch >40:
        lr *= 1e-7
    elif epoch > 30:
        lr *= 1e-4
    elif epoch > 20:
        lr *= 1e-2
    elif epoch >10:
        lr *= 1e-1
    print('Learning rate:', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

# define loss, metrics, optimizer
optimizer = Adam(lr = lr_schedule(0) )
#optimizer = SGD(lr = lr_schedule(0), momentum=0.9, nesterov = True)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Learning rate: 1e-05
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 32)   2432        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 32)   0           batch_normalization[0][0]        
________________________________________________________________________________________

In [7]:
batch_size = 128
epc = 100

start = time.time()

# training
train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
step_size_train = train_generator.n // train_generator.batch_size

# validation
val_generator = val_datagen.flow(x_val, y_val, batch_size=batch_size)
step_size_val = val_generator.n // val_generator.batch_size

# test
test_generator = test_datagen.flow(x_test, y_test, batch_size=batch_size)
step_size_test = test_generator.n // test_generator.batch_size

# usefull callbacks
log_dir="Logs/92full"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_reducer = ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=5, verbose=1)
early_stopper = EarlyStopping(monitor='val_acc', patience=15, verbose=1)

model.fit_generator(train_generator,
                    steps_per_epoch = step_size_train,
                    epochs = epc,
                    validation_data = val_generator,
                    validation_steps = step_size_val,
                    callbacks=[tensorboard_callback, lr_reducer, lr_scheduler, early_stopper])

end = time.time()
print("Time taken by above cell is {}.".format((end-start)/60))

Learning rate: 1e-05
Epoch 1/100
351/351 [==============================] - 275s 782ms/step - loss: 2.9477 - acc: 0.2414 - val_loss: 1.9207 - val_acc: 0.3081
Learning rate: 1e-05
Epoch 2/100
351/351 [==============================] - 206s 586ms/step - loss: 1.9447 - acc: 0.3099 - val_loss: 1.8334 - val_acc: 0.3506
Learning rate: 1e-05
Epoch 3/100
351/351 [==============================] - 206s 586ms/step - loss: 1.8220 - acc: 0.3421 - val_loss: 1.8121 - val_acc: 0.3618
Learning rate: 1e-05
Epoch 4/100
351/351 [==============================] - 206s 586ms/step - loss: 1.7469 - acc: 0.3658 - val_loss: 1.7857 - val_acc: 0.3778
Learning rate: 1e-05
Epoch 5/100
351/351 [==============================] - 205s 584ms/step - loss: 1.6949 - acc: 0.3859 - val_loss: 1.7035 - val_acc: 0.4020
Learning rate: 1e-05
Epoch 6/100
351/351 [==============================] - 205s 583ms/step - loss: 1.6438 - acc: 0.4038 - val_loss: 1.6216 - val_acc: 0.4127
Learning rate: 1e-05
Epoch 7/100
351/351 [==========

In [8]:
# evaluation
val_scores = model.evaluate_generator(val_generator, verbose=0)
test_scores = model.evaluate_generator(test_generator, verbose=0)
print('validation loss:', val_scores[0])
print('validation accuracy:', val_scores[1])
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

validation loss: 1.3192960618972778
validation accuracy: 0.5274
Test loss: 1.3441968976974488
Test accuracy: 0.5181


In [9]:
model.save('Model92_paper.h5')  # 0.7418

In [11]:
mv 20191211-005015/ 56_NAL/

In [19]:
cd ..

/content/gdrive/My Drive/Deep-Learning
